# Import

In [ ]:
import random
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Optional

import cv2
import torch
import torch.nn.functional as F
from PIL import Image
from tqdm import tqdm
from transformers import ViTForImageClassification, ViTImageProcessor
from transformers import SwinForImageClassification
from torchvision import transforms



# Settings

In [51]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [52]:
# 트래이닝 형태 / 검증 형태에 따른 전처리 정의
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),
])

In [89]:
MODEL_ID = "microsoft/swin-base-patch4-window7-224"
TEST_DIR = Path("./test_data/cropdata")  # test 데이터 경로

# Submission
OUTPUT_DIR = Path("./output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)  # output 폴더 없으면 생성

OUT_CSV = OUTPUT_DIR / "baseline_submission.csv"

In [ ]:
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".jfif"}
VIDEO_EXTS = {".mp4", ".mov"}

TARGET_SIZE = 224
NUM_FRAMES = 10  # 비디오 샘플링 프레임 수

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")

Device: cpu


# Utils

In [55]:
def uniform_frame_indices(total_frames: int, num_frames: int) -> np.ndarray:
    """비디오 프레임을 균등하게 샘플링"""
    if total_frames <= 0:
        return np.array([], dtype=int)
    if total_frames <= num_frames:
        return np.arange(total_frames, dtype=int)
    return np.linspace(0, total_frames - 1, num_frames, dtype=int)

def get_full_frame_padded(pil_img: Image.Image, target_size=(224, 224)) -> Image.Image:
    """전체 이미지를 비율 유지하며 정사각형 패딩 처리"""
    img = pil_img.convert("RGB")
    img.thumbnail(target_size, Image.BICUBIC)
    new_img = Image.new("RGB", target_size, (0, 0, 0))
    new_img.paste(img, ((target_size[0] - img.size[0]) // 2,
                        (target_size[1] - img.size[1]) // 2))
    return new_img

def read_rgb_frames(file_path: Path, num_frames: int = NUM_FRAMES) -> List[np.ndarray]:
    """이미지 또는 비디오에서 RGB 프레임 추출"""
    ext = file_path.suffix.lower()
    
    # 이미지 파일
    if ext in IMAGE_EXTS:
        try:
            img = Image.open(file_path).convert("RGB")
            return [np.array(img)]
        except Exception:
            return []
    
    # 비디오 파일
    if ext in VIDEO_EXTS:
        cap = cv2.VideoCapture(str(file_path))
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if total <= 0:
            cap.release()
            return []
        
        frame_indices = uniform_frame_indices(total, num_frames)
        frames = []
        
        for idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
            ret, frame = cap.read()
            if not ret:
                continue
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        cap.release()
        return frames
    
    return []

# Data Preprocessing

In [56]:
import numpy as np
from PIL import Image
import mediapipe as mp

mp_face = mp.solutions.face_detection

def _clamp(v, lo, hi):
    return max(lo, min(hi, v))

def crop_face_mediapipe(
    img: Image.Image,
    min_conf: float = 0.5,
    margin: float = 0.25,     # 얼굴 박스 주변 여유
    make_square: bool = True,
    fill_color=(0, 0, 0)      # 박스가 이미지 밖으로 나가면 패딩 색
) -> Image.Image:
    """
    얼굴 1개(가장 큰 박스) 크롭. 실패 시 원본 반환.
    """
    img = img.convert("RGB")
    w, h = img.size
    rgb = np.array(img)  # RGB

    # mediapipe는 RGB 입력 사용
    with mp_face.FaceDetection(model_selection=0, min_detection_confidence=min_conf) as fd:
        res = fd.process(rgb)

    if not res.detections:
        return img  # 실패 시 원본

    # 가장 큰 얼굴 선택
    best = None
    best_area = -1
    for det in res.detections:
        bbox = det.location_data.relative_bounding_box
        x1 = int(bbox.xmin * w)
        y1 = int(bbox.ymin * h)
        bw = int(bbox.width * w)
        bh = int(bbox.height * h)
        x2 = x1 + bw
        y2 = y1 + bh
        area = max(0, bw) * max(0, bh)
        if area > best_area:
            best_area = area
            best = (x1, y1, x2, y2)

    x1, y1, x2, y2 = best

    # margin 적용
    bw = x2 - x1
    bh = y2 - y1
    mx = int(bw * margin)
    my = int(bh * margin)
    x1 -= mx; y1 -= my; x2 += mx; y2 += my

    # 정사각형으로 확장
    if make_square:
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2
        half = max(x2 - x1, y2 - y1) // 2
        x1, x2 = cx - half, cx + half
        y1, y2 = cy - half, cy + half

    # 이미지 밖으로 나가면 패딩 후 크롭
    pad_l = max(0, -x1)
    pad_t = max(0, -y1)
    pad_r = max(0, x2 - w)
    pad_b = max(0, y2 - h)

    if pad_l or pad_t or pad_r or pad_b:
        new_w = w + pad_l + pad_r
        new_h = h + pad_t + pad_b
        canvas = Image.new("RGB", (new_w, new_h), fill_color)
        canvas.paste(img, (pad_l, pad_t))
        x1 += pad_l; x2 += pad_l
        y1 += pad_t; y2 += pad_t
        return canvas.crop((x1, y1, x2, y2))

    # 그냥 크롭
    x1 = _clamp(x1, 0, w)
    y1 = _clamp(y1, 0, h)
    x2 = _clamp(x2, 0, w)
    y2 = _clamp(y2, 0, h)
    if x2 <= x1 or y2 <= y1:
        return img

    return img.crop((x1, y1, x2, y2))

In [57]:
## 비디오 프래임 점수 집계 방식 // max, mean, topkmean 지원
def aggregate_frame_probs(
    probs: torch.Tensor,
    method: str = "mean",
    topk: int = 3
) -> float:
    """
    probs: shape [T] (각 프레임의 fake 확률)
    method: "mean" | "max" | "topkmean"
    """
    if probs.numel() == 0:
        return 0.0

    method = method.lower()

    if method == "mean":
        return float(probs.mean().item())

    if method == "max":
        return float(probs.max().item())

    if method in ("topk", "topkmean", "top-k", "top-k-mean"):
        k = int(topk)
        k = max(1, min(k, probs.numel()))
        topk_vals = torch.topk(probs, k=k, largest=True).values
        return float(topk_vals.mean().item())

    raise ValueError(f"Unknown method: {method}. Use 'mean', 'max', or 'topkmean'.")

In [58]:
from typing import Callable


class PreprocessOutput:
    def __init__(
        self,
        filename: str,
        imgs: List[Image.Image],
        error: Optional[str] = None
    ):
        self.filename = filename
        self.imgs = imgs
        self.error = error

def preprocess_one(file_path: Path,transform: Callable,num_frames: int = NUM_FRAMES) -> PreprocessOutput:
    """
    파일 하나에 대한 전처리 수행
    
    Args:
        file_path: 처리할 파일 경로
        num_frames: 비디오에서 추출할 프레임 수
    
    Returns:
        PreprocessOutput 객체
    """
    try:
        frames = read_rgb_frames(file_path, num_frames=num_frames)
              
        imgs: List[Image.Image] = []
        
        for rgb in frames:
            pil = Image.fromarray(rgb).convert("RGB")
            # 🔹 얼굴 크롭 시도 (실패하면 원본 반환)
            pil = crop_face_mediapipe(pil, min_conf=0.5, margin=0.25)
            x = transform(pil)     # ✅ 여기서 Swin 전처리 적용 (Tensor로 변환)
            imgs.append(x)
        
        return PreprocessOutput(file_path.name, imgs, None)
    
    except Exception as e:
        return PreprocessOutput(file_path.name, [], str(e))

# Model Load

In [ ]:
print("Loading model...")

model = SwinForImageClassification.from_pretrained(
    MODEL_ID,
    num_labels=1,                 # ✅ binary (real/fake)
    ignore_mismatched_sizes=True  # ✅ head 크기 달라도 로드
).to(DEVICE)

model.train()   # 🔴 파인튜닝이므로 train()

print(f"Model loaded: {MODEL_ID}")
print(f"num_labels = {model.config.num_labels}")

Loading model...
Model loaded: prithivMLmods/Deep-Fake-Detector-v2-Model
Model config: num_labels=2
id2label: {0: 'Realism', 1: 'Deepfake'}


In [60]:
from pathlib import Path
from PIL import Image

inp = Path("./test_data/cropdata")     # 여기에 사진들 넣기
out = Path("./test_data/croptestdata")  # 결과물 저장 경로
out.mkdir(exist_ok=True)

for p in inp.glob("*"):
    if p.suffix.lower() not in [".jpg",".jpeg",".png",".jfif"]:
        continue
    img = Image.open(p).convert("RGB")
    face = crop_face_mediapipe(img, min_conf=0.5, margin=0.25)
    face.save(out / p.name)

print("done:", out)


done: test_data\croptestdata


In [61]:
@torch.no_grad()
def infer_file_score(
    file_path: Path,
    model,
    transform,
    device: str,
    agg: str = "mean",
    topk: int = 3,
    num_frames: int = NUM_FRAMES
) -> float:
    out = preprocess_one(file_path, transform=transform, num_frames=num_frames)

    if out.error or not out.imgs:
        return 0.0

    xs = torch.stack(out.imgs, dim=0).to(device)  # [T,3,224,224]

    model.eval()
    outputs = model(xs)

    # ✅ logits 추출 (HF ImageClassifierOutput / tuple / tensor 모두 대응)
    if hasattr(outputs, "logits"):
        logits = outputs.logits
    elif isinstance(outputs, (tuple, list)):
        logits = outputs[0]
    else:
        logits = outputs  # torch.Tensor

    logits = logits.squeeze()

    # logits shape 정리: [T]
    if logits.dim() == 0:
        logits = logits.unsqueeze(0)
    elif logits.dim() > 1:
        logits = logits.view(-1)

    probs = torch.sigmoid(logits).detach().cpu()  # [T]
    return aggregate_frame_probs(probs, method=agg, topk=topk)


In [74]:

fp = Path("./test_data/cropdata/real_0.jpg")
score_mean = infer_file_score(fp, model, val_transform, DEVICE, agg="mean")
score_max  = infer_file_score(fp, model, val_transform, DEVICE, agg="max")
score_top3 = infer_file_score(fp, model, val_transform, DEVICE, agg="topkmean", topk=3)

print(score_mean, score_max, score_top3)

fp1 = Path("./test_data/cropdata/abarnvbtwb.mp4")
score_mean = infer_file_score(fp1, model, val_transform, DEVICE, agg="mean", num_frames=10)
score_max  = infer_file_score(fp1, model, val_transform, DEVICE, agg="max", num_frames=10)
score_top3 = infer_file_score(fp1, model, val_transform, DEVICE, agg="topkmean", topk=3, num_frames=10)

print(score_mean, score_max, score_top3)



0.5226365923881531 0.5984196662902832 0.5226365923881531
0.5174304842948914 0.7963025569915771 0.7775881886482239


In [63]:
def infer_fake_probs(pil_images: List[Image.Image]) -> List[float]:
    if not pil_images:
        return []

    probs: List[float] = []

    with torch.inference_mode():
        inputs = processor(images=pil_images, return_tensors="pt")
        inputs = {k: v.to(DEVICE, non_blocking=True) for k, v in inputs.items()}
        logits = model(**inputs).logits
        batch_probs = F.softmax(logits, dim=1)[:, 1]
        probs.extend(batch_probs.cpu().tolist())

    return probs

# Inference

In [93]:
files = sorted([p for p in TEST_DIR.iterdir() if p.is_file()])
print(f"Test data length: {len(files)}")

results: Dict[str, float] = {}

# 전처리 및 추론
for file_path in tqdm(files, desc="Processing"):
    out = preprocess_one(file_path)
    
    # 1. 에러 로깅
    if out.error:
        print(f"[WARN] {out.filename}: {out.error}")
    
    # 2. 정상 추론
    elif out.imgs:
        probs = infer_fake_probs(out.imgs)
        results[out.filename] = float(np.mean(probs)) if probs else 0.0
    
    # 3. 둘 다 없으면 0.0 (real)
    else:
        results[out.filename] = 0.0

print(f"Inference completed. Processed: {len(results)} files")

Test data length: 2


Processing:   0%|          | 0/2 [00:00<?, ?it/s]


TypeError: preprocess_one() missing 1 required positional argument: 'transform'

# Submission

In [ ]:

AGG_METHOD = "topkmean"   # "mean" | "max" | "topkmean"
TOPK = 3

files = sorted([p for p in TEST_DIR.rglob("*") if p.is_file()])
rows = []

for fp in tqdm(files, desc="Inference"):
    if fp.suffix.lower() not in (IMAGE_EXTS | VIDEO_EXTS):
        continue

    prob = infer_file_score(
        fp, model, val_transform, DEVICE,
        agg=AGG_METHOD, topk=TOPK, num_frames=NUM_FRAMES
    )

    # 혹시 모를 안전장치 (0~1로 클램프)
    prob = float(max(0.0, min(1.0, prob)))

    rows.append({"filename": fp.name, "prob": prob})

df = pd.DataFrame(rows)
df.to_csv(OUT_CSV, index=False)
print("saved:", OUT_CSV)
print(df.head())


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\Kim\HAI\.venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
100%|██████████| 2/2 [00:12<00:00,  6.11s/it]

saved: output\baseline_submission.csv
         filename      mean       max      top3
0  abarnvbtwb.mp4  0.517430  0.796303  0.777588
1      real_0.jpg  0.522637  0.598420  0.522637
